In [28]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.common.by import By
import pandas as pd
from datetime import datetime, timedelta
import selenium

from src import scrapers

# Path to geckodriver executable
geckodriver_path = '/snap/bin/firefox.geckodriver'


s = Service(executable_path=geckodriver_path)
# Create a Firefox webdriver instance
# opens a window
driver = webdriver.Firefox(service=s)

In [29]:
class UolNewsScraper():
    """Scraper for the G1 news website"""

    def __init__(self, n_scrolls = 10):
        self.url = 'https://noticias.uol.com.br/?clv3=true'
        
        # how many times to scrool the page
        self.n_scrolls = n_scrolls
        
    def _scroll_page(self, driver: selenium.webdriver):
        """Scroll the page by n_scrolls iterations"""
        
        current_height = driver.execute_script("return document.body.scrollHeight")

        for i in range(self.n_scrolls):
            # scroll to the end of the page
            driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
            sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            
            # if not autoload
            if new_height == current_height:
                # click the 'Veja mais' button
                driver.find_element(By.CSS_SELECTOR, value='.btn-search').click()
                
                # repeat the scroll
                driver.execute_script(f"window.scrollTo(0,document.body.scrollHeight)")
                current_height = driver.execute_script("return document.body.scrollHeight")
            else:
                current_height = new_height
                
            print(f'Scroll {i + 1}, height={current_height}')
            
            # time for the page to load
            sleep(4)
            
        
    def _get_scraped_news(self, driver: selenium.webdriver):
        """Obtain the scraped news from the website"""
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'lxml')
        
        content = soup.find('section', class_ = 'latest-news')
        news_list = content.find_all('div', class_ = 'thumb-caption')
        
        
        titles = []
        times = []
        resumes = []

        for news in news_list:
            # obtain the info of a individual news
            title = news.find('h3', class_ = 'thumb-title')
            titles.append(title.text)

            time = news.find('time', class_ = 'thumb-date')
            times.append(None if time is None else time.text)

            resume = news.find('p', class_ = 'thumb-description')
            resumes.append(None if resume is None else resume.text)

        # create the final dataframe
        news_df = pd.DataFrame({
            'Title': titles,
            'Time': times,
            'Resume': resumes
        })     

        # create the final dataframe
        news_df = pd.DataFrame({
            'Title': titles,
            'Time': times,
            'Resume': resumes
        })     
        
        return news_df
    
    def _convert_to_datetime(self, time_str: str) -> datetime:
        """convert "Há X [time unit]" to datetime"""
        if 'hora' in time_str or 'horas' in time_str:
            hours_ago = int(time_str.split(' ')[1])
            return datetime.now() - timedelta(hours=hours_ago)
        
        elif 'minuto' in time_str or 'minutos' in time_str:
            minutes_ago = int(time_str.split(' ')[1])
            return datetime.now() - timedelta(minutes=minutes_ago)
        
        elif 'dia' in time_str or 'dias' in time_str:
            days_ago = int(time_str.split(' ')[1])
            return datetime.now() - timedelta(days=days_ago)
        
        elif 'mês' in time_str or 'meses' in time_str:
            months_ago = int(time_str.split(' ')[1])
            # Note: This is an approximation as timedelta does not support months directly
            return datetime.now() - timedelta(days=30*months_ago)
        else:
            return None  # If the format does not match, return None
        
    
    def _data_cleaning(self, news_df: pd.DataFrame):
        """Do simple data cleaning after the scrap"""
        news_df['Time'] = news_df['Time'].apply(self._convert_to_datetime)
        return news_df


    def scrap_news(self, driver: selenium.webdriver) -> pd.DataFrame:
        """Scrap the news for the G1 website

        Parameters
        ----------
        driver : selenium.webdriver
            Current webdriver

        Returns
        -------
        pd.DataFrame
            The news scraped
        """
        driver.get(self.url)
        sleep(1)
        
        self._scroll_page(driver)
        
        news_df = self._get_scraped_news(driver)
        
        # news_df = self._data_cleaning(news_df)
        
        return news_df

In [30]:
scraper = UolNewsScraper()

In [31]:
scraper.scrap_news(driver)

Scroll 1, height=9931
Scroll 2, height=11360
Scroll 3, height=12641
Scroll 4, height=12658
Scroll 5, height=13773
Scroll 6, height=13855
Scroll 7, height=13855
Scroll 8, height=15023
Scroll 9, height=16266
Scroll 10, height=16313


,Title,Time,Resume
0,Barulho interrompe comício de Trump; ex-presid...,13/07/2024 19h39,None
1,Polícia do Quênia encontra novos corpos desmem...,13/07/2024 19h32,A polícia do Quênia anunciou neste sábado (13)...
2,Trump é retirado de comício na Pensilvânia com...,13/07/2024 19h27,Trump é retirado de comício na Pensilvânia com...
3,Polícia investiga assassinato de assessor de M...,13/07/2024 19h23,"O assessor do MC Livinho, Adileon Eva dos Sant..."
4,Comício de Trump é interrompido após sons de a...,13/07/2024 19h16,None
5,"Alcaraz-Djokovic, revanche e duelo de gerações...",13/07/2024 19h14,"O espanhol Carlos Alcaraz, de 21 anos, que con..."
6,Idosa fratura os dois braços após colisão fron...,13/07/2024 19h10,Uma idosa de 70 anos ficou ferida na tarde des...
7,"Lula pesca na Granja do Torto, e Bolsonaro and...",13/07/2024 19h08,"O sábado, 13, foi de diversão e fuga da rotina..."
8,Maior apreensão de cigarros contrabandeados do...,13/07/2024 18h54,A Polícia Federal e a Policia Militar de São P...
9,"Lula pesca na Granja do Torto, e Bolsonaro and...",13/07/2024 19h08,"O sábado, 13, foi de diversão e fuga da rotina..."


In [17]:

content

<section class="latest-news"><div class="section-title" uolbpack-initialized="true"> <hr class="thin-normal"/> <div class="container"><div class="row"><div class="col-xs-8 col-sm-13 col-md-15 col-lg-16"> <div class="section-info"> <h3 class="heading-style"> <a data-audience-click='{"reference":"titulo-modulo","component":"results-index-click","mediaName":"Home","mediaTitle":"Home noticias"}' href="https://noticias.uol.com.br/ultimas/"><span class="custom-title">Últimas notícias<svg><use xlink:href="#svg-single-arrow-right"></use></svg></span></a> </h3> </div></div> </div> </div></div><div class="container"><section class="results-index gallery-list" uolbpack-initialized="true"><script type="application/ld+json">[{"@type":"ItemList","@context":"http://schema.org","numberOfItems":9,"itemListElement":[[{"@type":"ListItem","name":"Barulho de tiros interrompe comício de Trump nos EUA","position":1,"url":"https://noticias.uol.com.br/internacional/ultimas-noticias/2024/07/13/trump-comicio.htm

In [26]:
titles = []
times = []
themes = []
headers = []
resumes = []

In [27]:


news_df

,Title,Time,Resume
0,Barulho de tiros interrompe comício de Trump n...,13/07/2024 19h39,None
1,Polícia do Quênia encontra novos corpos desmem...,13/07/2024 19h32,A polícia do Quênia anunciou neste sábado (13)...
2,Trump é retirado de comício na Pensilvânia com...,13/07/2024 19h27,Trump é retirado de comício na Pensilvânia com...
3,Polícia investiga assassinato de assessor de M...,13/07/2024 19h23,"O assessor do MC Livinho, Adileon Eva dos Sant..."
4,Comício de Trump é interrompido após sons de a...,13/07/2024 19h16,None
...,...,...,...
64,Polícia Federal realiza em SP maior apreensão ...,13/07/2024 16h01,A apreensão equivale a 20 carretas lotadas e s...
65,'Noite vira dia' no nordeste brasileiro após p...,13/07/2024 15h59,A queda de um meteoro proporcionou uma vista ú...
66,Meteoro? objeto luminoso atravessa o céu de Pi...,13/07/2024 15h57,Um objeto luminoso cruzou o céu de cidades do ...
67,Netanyahu diz não ter certeza de que líder do ...,13/07/2024 15h55,JERUSALÉM (Reuters) - O primeiro-ministro ...
